<a href="https://colab.research.google.com/github/HackElite-FYP/Legal-Research-Platform-Core/blob/main/colab-main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata

GH_UNAME = userdata.get('GH_UNAME')
GH_APIKEY = userdata.get('GH_APIKEY')
GH_EMAIL = userdata.get('GH_EMAIL')
PRIMARY_REPO_NAME = 'Legal-Research-Platform'
LOCAL_REPO_DIR = '/content/drive/MyDrive/FYP/GitHub/Legal-Research-Platform'

!git config --global user.name "{GH_UNAME}"
!git config --global user.email "{GH_EMAIL}"

%cd {LOCAL_REPO_DIR}

/content/drive/MyDrive/FYP/GitHub/Legal-Research-Platform


In [2]:
!git fetch

!git pull

Already up to date.


In [3]:
!git add .

!git status

!git commit -m 'initialize with colab'

# !git push

On branch main
Your branch is up to date with 'origin/main'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	modified:   colab-main.ipynb

[main efb13b2] initialize with colab
 1 file changed, 1 insertion(+), 87 deletions(-)
 rewrite colab-main.ipynb (100%)
